# Ebpf Tutorial

We are going to explore the following questions:
1. Why do we care about ebpf?
2. What is ebpf?
3. How did it become what it is?
4. Okay so now what?

Before all of that I'll run you through a bit of setup.

## Setup
1. ssh into the shell machine `ssh -L <port-id>:127.0.0.1:<port-id> -i <private-key> <username>@<machine>`.
2. Once in your own shell clone the ebpf-tutorial `git clone https://github.com/ldos-project/ebpf-tutorial.git`.
3. Move into that directory `cd ebpf-tutorial`.
4. Now run `pipenv install && pipenv shell`. This should use pipenv to install dependencies to control ebpf.
5. Run this so jupyter can see the bpf compiler ``export JUPYTER_PATH=$(dirname `find /usr/lib -name bcc`):$PYTHONPATH``
6. Finally let's open this notebook by running: `JUPYTER_PORT=<port-id> jupyter notebook`. You should be able to connect by using the URL that it provides in your browser.

## Why do we care about ebpf?
- Why do y'all think we care about this?
- What have you heard?
Let's use a running example of a program.

### Simple example program
We just cat out a file.

In [1]:
import subprocess
proc = subprocess.run(["cat", "install_script.sh"])

sudo apt install pipenv -y
sudo apt install bpftrace -y


Recall that in this project we are interested in exactly how all of these things are happening at the lowest levels.
Using just userspace style tools here is the first tool I often try:

In [2]:
import subprocess
proc = subprocess.run(["strace", "cat", "install_script.sh"])

sudo apt install pipenv -y
sudo apt install bpftrace -y


execve("/usr/bin/cat", ["cat", "install_script.sh"], 0x7ffd7d3a7998 /* 50 vars */) = 0
brk(NULL)                               = 0x5615625d3000
arch_prctl(0x3001 /* ARCH_??? */, 0x7fff435933d0) = -1 EINVAL (Invalid argument)
mmap(NULL, 8192, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_ANONYMOUS, -1, 0) = 0x7fafe09e3000
access("/etc/ld.so.preload", R_OK)      = -1 ENOENT (No such file or directory)
openat(AT_FDCWD, "/home/adityat/.local/lib/glibc-hwcaps/x86-64-v3/libc.so.6", O_RDONLY|O_CLOEXEC) = -1 ENOENT (No such file or directory)
newfstatat(AT_FDCWD, "/home/adityat/.local/lib/glibc-hwcaps/x86-64-v3", 0x7fff435925f0, 0) = -1 ENOENT (No such file or directory)
openat(AT_FDCWD, "/home/adityat/.local/lib/glibc-hwcaps/x86-64-v2/libc.so.6", O_RDONLY|O_CLOEXEC) = -1 ENOENT (No such file or directory)
newfstatat(AT_FDCWD, "/home/adityat/.local/lib/glibc-hwcaps/x86-64-v2", 0x7fff435925f0, 0) = -1 ENOENT (No such file or directory)
openat(AT_FDCWD, "/home/adityat/.local/lib/tls/haswell/x86_64/libc.

1. What insights can we gain from this data?
1. What are the limitations of accepting data this way?
2. Should we just throw this into an LLM?
3. What data is missing?

### Example with ebpf
We will now use ebpf to extract information about openat.
There is a simple tool that uses it called `bpftrace` that I will show, this is a high level tool that uses ebpf under the hood.

*Show bpftrace*

I am far more partial to the real thing because it allows for *clean data*, with the least overhead.

In [12]:
%%writefile capture_openat.py
from bcc import BPF
bpf_txt = """
#include <linux/sched.h>
BPF_PERF_OUTPUT(openat_output);
BPF_HASH(openat_hash, u32, u64);

typedef struct output_data {
    u64 tgid;
    u64 start_ns;
    u64 end_ns;
} output_data_t;

int kprobe__do_openat(struct pt_regs* ctx, int dfd, const char __user *filename,
			   struct open_how *how) {
    u32 tgid = bpf_get_current_pid_tgid() >> 32;
    u64 ts = bpf_ktime_get_ns();
    openat_hash.insert(&tgid, &ts);
    return 0;
}
int kretprobe__do_openat(struct pt_regs* ctx) {
    u64 end_ts = bpf_ktime_get_ns();

    u32 tgid = bpf_get_current_pid_tgid() >> 32;\
    u64* start_ts = openat_hash.lookup(&tgid);
    if(start_ts == NULL) return 0;
    output_data_t data;
    data.tgid = tgid;
    data.start_ns = *start_ts;
    data.end_ns = end_ts;
    openat_hash.delete(&tgid);
    return 0;
}
"""

bpf = BPF(text = bpf_txt)
bpf.attach_kprobe(event=b"do_sys_openat2", fn_name="kprobe__do_openat")
bpf.attach_kretprobe(event=b"do_sys_openat2", fn_name="kretprobe__do_openat")

def print_event(cpu, data, size):
    output = bpf["openat_output"].event(data)
    print(f"tgid:\t{output.tgid}, elapsed:\t{output.end_ns - output.start_ns}")
    
bpf["openat_output"].open_perf_buffer(print_event)

while 1:
    try:
        bpf.perf_buffer_poll()
    except KeyboardInterrupt:
        print()
        exit()

Overwriting capture_openat.py


In [21]:
import subprocess
import signal
os.environ["PYTHONPATH"] = "/usr/lib/python3/dist-packages"
blah = subprocess.check_output("sudo python3 capture_openat.py", shell=True)
proc = subprocess.run(["cat", "install_script.sh"])
blah.kill(signal.SIGINT)
print(blah)

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


CalledProcessError: Command 'sudo python3 capture_openat.py' returned non-zero exit status 1.